# Set Directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
directory = '/conmtent/drive/MyDrive/NLP_Project/data/raw'
if not path.exists(directory):
  os.mkdir(directory)

# Scrape animeseries.io

### Packages

In [ ]:
import urllib
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs
import re
from string import ascii_uppercase
import csv
import time
import pandas as pd

### Website Scrapping

In [ ]:
base_url = "https://www4.animeseries.io" #initilizing base url
url_template = "https://www4.animeseries.io/search/character=special" #initilizing url template for alphabetical search
url = base_url

In [ ]:
titles = []
descriptions = []
all_tags = []

#creating a list of charecters to loop through
charecters = list(ascii_uppercase)
charecters.insert(0,"special")


for alphabet in tqdm(charecters[:], "Scrapping animeseries.io"):

    #substituting 'special' keyword with alphabet
    url = re.sub('special', alphabet, url_template)
    
    #Requesting for url using Request Library
    url_link = Request(url, headers={'User-Agent': 'Mozilla/5.0'}) 
    url_page = urlopen(url_link).read() 
    url_html = bs(url_page, 'html.parser') 

    items = url_html.find_all('div',class_='content_episode revent datagrild')[0]

    html_links = items.find_all('a') 

    for link in html_links:

      html_link = link.get('href')

      new_url = base_url + html_link

      url_link = Request(new_url, headers={'User-Agent': 'Mozilla/5.0'})
      url_page = urlopen(url_link).read()
      url_html = bs(url_page, 'html.parser')


      anime_containers = url_html.find('div',class_="content")
      contents = anime_containers.find('div',class_='right')

      # Title
      try:
        titles.append(contents.h1.text)
      except:
        titles.append(None)

      # Description
      try:
          descriptions.append(contents.p.text)
      except: 
          descriptions.append(None)

      # Tags
      try:
        tag = contents.find_all('p', class_='des')[-1]
        tags = tag.text.strip().split('\n')[1:]
        all_tags.append(tags)
      except:  
        all_tags.append(None)

In [ ]:
a = {'Title': titles, 'Description' : descriptions, 'Tags' : all_tags,}
df = pd.DataFrame.from_dict(a, orient='index')
df = df.transpose()

In [ ]:
df.to_csv('anime-series.csv')

In [ ]:
df.head(20)

,Title,Description,Tags
0,?Maria-sama ga Miteru Haru 2nd,Life goes on for the ladies in the Yamayurikai...,"[Drama, School, Shounen Ai, Slice of Life]"
1,.Hack//G.U. Returner,hack//G.U. Returner is a single-episode OVA of...,"[Adventure, Drama, Game, Harem, Martial Arts, ..."
2,.hack//G.U. Trilogy,"Based on the CyberConnect2 HIT GAME, now will ...","[Action, Game, Harem, Seinen]"
3,.hack//Gift,As an expression of gratitude for the heroes o...,"[Comedy, Fantasy, Game]"
4,.hack//Legend of the Twilight,.hack//Legend of the Twilight,"[Adventure, Comedy, Fantasy, Sci-Fi, Shounen]"
5,.hack//Liminality,"The story follows three girls, Mai Minase, Yuk...","[Mystery, Sci-Fi]"
6,.hack//Quantum,"Plot Summary: Tobias, Mary, and Sakuya challen...","[Action, Adventure, Fantasy, Sci-Fi]"
7,.hack//Roots,hack//Roots is the prequel to the .hack// vide...,"[Adventure, Drama, Fantasy, Game, Sci-Fi]"
8,.hack//Roots (Dub),"After the destruction of ""The World"" in 2015, ...","[Adventure, Drama, Fantasy, Game, Sci-Fi]"
9,.Hack//Sign,Taking place 2 years after the “Pluto’s Kiss” ...,"[Adventure, Fantasy, Magic, Mystery, Sci-Fi]"


### Save data

In [ ]:
!cp /content/anime-series.csv /content/drive/MyDrive/NLP_Project/data/raw

In [ ]:
# df.count

<bound method DataFrame.count of                                                   Title  ...                                               Tags
0                        ?Maria-sama ga Miteru Haru 2nd  ...         [Drama, School, Shounen Ai, Slice of Life]
1                                  .Hack//G.U. Returner  ...  [Adventure, Drama, Game, Harem, Martial Arts, ...
2                                   .hack//G.U. Trilogy  ...                      [Action, Game, Harem, Seinen]
3                                           .hack//Gift  ...                            [Comedy, Fantasy, Game]
4                         .hack//Legend of the Twilight  ...      [Adventure, Comedy, Fantasy, Sci-Fi, Shounen]
...                                                 ...  ...                                                ...
8795                  Zone of the Enders: Dolores (Dub)  ...  [Action, Adventure, Comedy, Mecha, Military, S...
8796                     Zone of the Enders: Dolores, I  ...  [Action, 

# Scrape Anime-Planet

### Packages

In [ ]:
import requests
from requests import get
from bs4 import BeautifulSoup
from html.parser import HTMLParser
import xml.sax.saxutils as saxutils
import html
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from tqdm import tqdm

### Website Scrapping

In [ ]:
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
    'referer':'https://www.google.com/'
}

#initiate data storage
titles = []
descriptions = []
anime_tags = []

pages = 457

for page in tqdm(range(1, pages+1), "Scrapping anime-planet"): 

  result = requests.get("https://www.anime-planet.com/anime/all?page="+ str(page) , headers=headers)
  soup = BeautifulSoup(saxutils.unescape(result.text), 'html.parser')

  sleep(randint(2,4))
  movie_div = soup.find_all('li', attrs={'data-type': 'anime', 'class':'card '})

  for div in movie_div:

    try:
      a = div.select('a')[0]
    except:
      continue


    # title
    try:
      title_section = a.get("title")
      title_parsed = BeautifulSoup(saxutils.unescape(title_section), 'html.parser')

      titles.append(title_parsed.h5.text)
    except:
      titles.append(None)
    

    # description
    try:   
      descriptions.append(title_parsed.p.text)
    except:
      descriptions.append(None)
    
    
    #tags
    try:
      tags = title_parsed.find('div', attrs={'class':"tags"}).ul
      all_tags = tags.find_all('li')
      all_tags = [x.text for x in all_tags]
      anime_tags.append(all_tags)

    except:
      anime_tags.append(None)



a = {'Title': titles, 'Description' : descriptions, 'Tags' : anime_tags,}
df = pd.DataFrame.from_dict(a, orient='index')
df = df.transpose()

Scrapping anime-planet: 100%|██████████| 457/457 [28:54<00:00,  3.80s/it]


In [ ]:
df.to_csv('anime-planet.csv')

In [ ]:
df.head(20)

,Title,Description,Tags
0,Attack on Titan The Final Season,It's been four years since the Scout Regiment ...,"[Action, Drama, Fantasy, Horror, Shounen, Dark..."
1,Fullmetal Alchemist: Brotherhood,The foundation of alchemy is based on the law ...,"[Action, Adventure, Drama, Fantasy, Mystery, S..."
2,Attack on Titan 3rd Season: Part II,The battle to retake Wall Maria begins now! Wi...,"[Action, Fantasy, Horror, Shounen, Dark Fantas..."
3,Jujutsu Kaisen,Although Yuji Itadori looks like your average ...,"[Action, Horror, Shounen, Curse, Monsters, Sch..."
4,Fruits Basket the Final,Final season of Fruits Basket.,"[Comedy, Drama, Fantasy, Romance, Shoujo, Anim..."
5,Haikyuu!! Karasuno High School vs Shiratorizaw...,"Picking up where the second season ended, the ...","[Shounen, Sports, Animeism, School Club, Schoo..."
6,A Silent Voice,"After transferring into a new school, a deaf g...","[Drama, Shounen, Disability, Melancholy, Menta..."
7,Demon Slayer: Kimetsu no Yaiba Movie - Mugen T...,Tanjirou and the group have completed their re...,"[Action, Drama, Fantasy, Shounen, Demons, Hist..."
8,Hunter x Hunter (2011),"Drawn to the mystique of the unknown, Hunters ...","[Action, Adventure, Drama, Fantasy, Shounen, M..."
9,your name.,Mitsuha and Taki are two total strangers livin...,"[Drama, Romance, Body Swapping, Gender Bender,..."


### Save Data

In [ ]:
!cp /content/anime-planet-v2.csv /content/drive/MyDrive/NLP_Project/data/raw

# Scrape Anime-Planet Manga

### Packages

In [ ]:
import requests
from requests import get
from bs4 import BeautifulSoup
from html.parser import HTMLParser
import xml.sax.saxutils as saxutils
import html
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from tqdm import tqdm

## Website Scrapping

In [ ]:
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
    'referer':'https://www.google.com/'
}

#initiate data storage
titles = []
descriptions = []
anime_tags = []

pages = 1763

for page in tqdm(range(1, pages+1), "Scrapping anime-planet manga"): 

  result = requests.get("https://www.anime-planet.com/manga/all?page="+ str(page) , headers=headers)
  soup = BeautifulSoup(saxutils.unescape(result.text), 'html.parser')

  sleep(randint(2,4))
  movie_div = soup.find_all('li', attrs={'data-type': 'manga', 'class':'card '})

  for div in movie_div:

    try:
      a = div.select('a')[0]
    except:
      continue


    # title
    try:
      title_section = a.get("title")
      title_parsed = BeautifulSoup(saxutils.unescape(title_section), 'html.parser')

      titles.append(title_parsed.h5.text)
    except:
      titles.append(None)
    

    # description
    try:   
      descriptions.append(title_parsed.p.text)
    except:
      descriptions.append(None)
    
    
    #tags
    try:
      tags = title_parsed.find('div', attrs={'class':"tags"}).ul
      all_tags = tags.find_all('li')
      all_tags = [x.text for x in all_tags]
      anime_tags.append(all_tags)

    except:
      anime_tags.append(None)


Scrapping anime-planet manga: 100%|██████████| 1763/1763 [1:54:27<00:00,  3.90s/it]


In [ ]:
a = {'Title': titles, 'Description' : descriptions, 'Tags' : anime_tags,}
df = pd.DataFrame.from_dict(a, orient='index')
df = df.transpose()

In [ ]:
df.to_csv('anime-planet-manga.csv')

In [ ]:
df.head(20)

,Title,Description,Tags
0,Solo Leveling,E-class hunter Jinwoo Sung is the weakest of t...,"[Action, Adventure, Fantasy, Manhwa, Webtoon, ..."
1,Flawless: Specials,This entry currently doesn't have a synopsis. ...,"[Comedy, Drama, Romance, Webtoon, Full Color, ..."
2,The Beginning After the End,"King Grey has unrivaled strength, wealth, and ...","[Action, Comedy, Fantasy, OEL, Webtoon, Full C..."
3,Fullmetal Alchemist,"The first law of alchemy is that ""one cannot g...","[Action, Adventure, Drama, Fantasy, Shounen, C..."
4,Berserk,Born beneath the gallows tree from which his d...,"[Action, Fantasy, Seinen, Dark Fantasy, Demons..."
5,Haikyuu!!,After losing his first and last volleyball mat...,"[Comedy, Drama, Shounen, Sports, School Life, ..."
6,Mo Dao Zu Shi (Novel),As the grandmaster who founded the Demonic Sec...,"[Action, Adventure, BL, Comedy, Drama, Fantasy..."
7,Movies Are Real,While working as a small supporting actor in a...,"[Action, Adventure, Drama, Fantasy, Manhwa, We..."
8,One Piece,Long ago the infamous Gol D. Roger was the str...,"[Action, Adventure, Comedy, Fantasy, Shounen, ..."
9,Eleceed,Jiwoo is a kind-hearted young man who harnesse...,"[Action, Comedy, Manhwa, Webtoon, Animal Trans..."


### Save Data

In [ ]:
!cp /content/anime-planet-manga.csv /content/drive/MyDrive/NLP_Project/data/raw

In [ ]:
df.count

<bound method DataFrame.count of                                                    Title  ...                                               Tags
0                                          Solo Leveling  ...  [Action, Adventure, Fantasy, Manhwa, Webtoon, ...
1                                     Flawless: Specials  ...  [Comedy, Drama, Romance, Webtoon, Full Color, ...
2                            The Beginning After the End  ...  [Action, Comedy, Fantasy, OEL, Webtoon, Full C...
3                                    Fullmetal Alchemist  ...  [Action, Adventure, Drama, Fantasy, Shounen, C...
4                                                Berserk  ...  [Action, Fantasy, Seinen, Dark Fantasy, Demons...
...                                                  ...  ...                                                ...
61674                                         Renchin! 2  ...  [BL, Drama, Romance, Yaoi, Adult Couples, Cook...
61675  Mamono na Shoujo ni Kakomareta kedo, Hatashite...  ...  